In [1]:
import numpy as np
import pims
import nd2
import pandas as pd
import xarray
import dask.array as da
import matplotlib.pyplot as pl

In [2]:
def readND2(filename):
    """
    
    """
    sequence = nd2.imread(filename, xarray = True, dask = True)
    frameT = sequence.metadata['experiment'][0].parameters.periodDiff.avg #this is the time per frame in ms
    scale = sequence.X[1] #this is the microns per pixel
    sequence['T'] = np.arange(len(sequence['T']))*frameT
    sequence.attrs = {'xyScale': float(scale),
                      'tScale': frameT}
    return(sequence)

In [3]:
seq = readND2('data/A1_s3001.nd2')
seq

<xarray.DataArray '_dask_block-7060f3470deffd836e17a4692376d751' (T: 5000,
                                                                  Y: 512, X: 512)>
dask.array<_dask_block, shape=(5000, 512, 512), dtype=uint16, chunksize=(1, 512, 512), chunktype=numpy.ndarray>
Coordinates:
  * Y        (Y) float64 0.0 0.1228 0.2456 0.3684 ... 62.38 62.51 62.63 62.75
  * X        (X) float64 0.0 0.1228 0.2456 0.3684 ... 62.38 62.51 62.63 62.75
  * T        (T) float64 0.0 10.01 20.02 30.02 ... 5.001e+04 5.002e+04 5.003e+04
Attributes:
    xyScale:  0.122802855328918
    tScale:   10.007511707419608

In [4]:
def readLIF(filename):
    """
    
    """
    lifSeq = pims.Bioformats(filename, read_mode = 'jpype')
    xscale = lifSeq.metadata.PixelsPhysicalSizeX(0)
    tscale = lifSeq.metadata.PlaneDeltaT(0,1)
    lifXCoords = np.arange(0., lifSeq.shape[1]*xscale, xscale)
    lifYCoords = np.arange(0., lifSeq.shape[2]*xscale, xscale)
    lifTCoords = np.arange(0., lifSeq.shape[0]*tscale, tscale)
    ds = da.from_array(lifSeq)
    sequence = xarray.DataArray(data = ds, dims = ["T", "Y", "X"], coords = (lifTCoords, lifYCoords, lifXCoords), attrs = dict(xyScale = xscale, tScale = tscale))
    return sequence

In [5]:
readLIF('data/21-03-31_ddm.lif')

<xarray.DataArray 'array-5e20a8405a8938d75e8344d06dd035f5' (T: 1000, Y: 1024,
                                                            X: 1024)>
dask.array<array, shape=(1000, 1024, 1024), dtype=uint8, chunksize=(500, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * T        (T) float64 0.0 0.012 0.024 0.036 0.048 ... 11.95 11.97 11.98 11.99
  * Y        (Y) float64 0.0 0.2045 0.409 0.6135 ... 208.6 208.8 209.0 209.2
  * X        (X) float64 0.0 0.2045 0.409 0.6135 ... 208.6 208.8 209.0 209.2
Attributes:
    xyScale:  0.20450566959921798
    tScale:   0.01200103759765625

In [18]:
def readTIF(filename, xscale = 1., tscale = 1.):
    """
    
    """
    tifSeq = pims.Bioformats(filename, read_mode = 'jpype')
    try:
        xscale = tifSeq.metadata.PixelsPhysicalSizeX(0)
    except:
        xscale = xs
    try:
        tscale = tifSeq.metadata.PixelsTimeIncrement(0)*1000.
    except:
        tscale = ts
    tifXCoords = np.arange(0., tifSeq.shape[1]*xscale, xscale)
    tifYCoords = np.arange(0., tifSeq.shape[2]*xscale, xscale)
    tifTCoords = np.arange(0., tifSeq.shape[0]*tscale, tscale)
    ds = da.from_array(tifSeq)
    sequence = xarray.DataArray(data = ds, dims = ["T", "Y", "X"], coords = (tifTCoords, tifYCoords, tifXCoords), attrs = dict(xyScale = xscale, tScale = tscale))
    return sequence

In [19]:
readTIF('data/A1_s3001.tif')

<xarray.DataArray 'array-f53a1e4610c7db682573f89bdb7587e4' (T: 5000, Y: 512,
                                                            X: 512)>
dask.array<array, shape=(5000, 512, 512), dtype=>u2, chunksize=(250, 256, 256), chunktype=numpy.ndarray>
Coordinates:
  * T        (T) float64 0.0 10.01 20.02 30.02 ... 5.001e+04 5.002e+04 5.003e+04
  * Y        (Y) float64 0.0 0.1228 0.2456 0.3684 ... 62.38 62.51 62.63 62.75
  * X        (X) float64 0.0 0.1228 0.2456 0.3684 ... 62.38 62.51 62.63 62.75
Attributes:
    xyScale:  0.12280285732776315
    tScale:   10.00751182436943

In [24]:
def read_file(filename, xs = None, ts = None):
    """
    """
    supported = ['.lif', '.nd2', '.tif', '.tiff']
    if filename[:-4] not in supported:
        print('Not a supported image format')
        return None
    if filename[:-4] == '.lif'
        readLIF(filename)
    elif filename[:-4] == 'nd2'
        readND2(filename)
    else:
        readTIF(filename, xscale = xs, tscale = ts)

In [25]:
readfile('x.png')

Not a supported image format
